In [ ]:
# pip install geopy

In [ ]:
# pip install geopandas

In [ ]:
# pip install folium

In [ ]:
import pandas as pd
import numpy as np
import folium
import time
import geopy
from geopy.extra.rate_limiter import RateLimiter
from folium.plugins import HeatMap
import matplotlib as mpl

l'equivalente NdomConParticelle in xlsx è DOM_Contratti_Attivi_20220929

DOM_Contratti_Attivi_20220929.tsv

In [ ]:
df = pd.read_csv(
    "C:/Users/gmauri01/Intellera/Progetto Fabbisogni Ta.Ri. SIR Gescan2 - Documenti/Dati/Analisi-Emesso-Non-Riscosso/dati-acquisiti/FornitureAMA/DOM_Contratti_Attivi_20220929.tsv",
    sep="\t",
)

In [ ]:
df.columns

In [ ]:
# m = folium.Map(location=[41.9109, 12.4818], tiles="cartodbpositron", zoom_start = 11)
m = folium.Map(location=[41.9109, 12.4818], zoom_start=11)
folium.GeoJson("C:/Users/gmauri01/intellera_code/progetto_comune_roma/municipi.geojson").add_to(m)
m

In [ ]:
locator = geopy.Nominatim(user_agent="myGeocoder")
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.01)
geocode = locator.geocode

In [ ]:
# start_time = time.time()
# bla=df['UTZ_VIA_DES'][0:100].apply(geocode)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
vie = df["UTZ_VIA_DES"].drop_duplicates().reset_index(drop=True)

In [ ]:
vie = vie + " ROMA"

In [ ]:
vie

Fatto così mi ci vogliono 10291 minuti, cioè 172 ore.
Se utilizzo 20 pc differenti o parallelizzo mi ci vogliono 8.6 ore.
Se lo fai con le risorse free di Google bisognerebbe spezzare in almeno 6 parti ciascun runtime sulle macchine. 
Daremmo a Google solo le vie e gli diremmo di trovare i CAP. 

Prepariamo intanto un modello giocattolo per poter mettere i punti sulla mappa. 1000 punti.

In [ ]:
# start_time = time.time()
# coord_vie=vie.apply(geocode)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# indirizzo=[]
# lat=[]
# long=[]
# for i in range(len(coord_vie)):
#     if coord_vie[i] is None :
#         print(i)
#         indirizzo.append(np.nan)
#         lat.append(np.nan)
#         long.append(np.nan)
#     else:
#         indirizzo.append(coord_vie[i][0])
#         lat.append(coord_vie[i][1][0])
#         long.append(coord_vie[i][1][1])

# indici da cercare
334
1241
1479
2445
2482
2528
2607
2706
3241
3322
4128
4202
4299
4625
4730
5278
5679
5783
5948
6197
6336
6447
6633
6763
6837
7142
7343
7713
8394
8687
8724
9073
9200
9231
9694
9720
9843
10012
10255
10453
10709
10905
11002
11262
11319
11487
11691
11950
12012
12426
12529
12612
12691
12754
12856
12938
13033
13036
13290
13323
13419
13465
13624

In [ ]:
# df=pd.DataFrame()
# df['vie']=vie
# df['indirizzo']=indirizzo
# df['lat']=lat
# df['long']=long
# df.to_csv('DISTINCT_vie.csv')

In [ ]:
# df.indirizzo.isnull().sum()

In [ ]:
data_vie = pd.read_csv("DISTINCT_vie.csv")

In [ ]:
data_vie = data_vie.drop(columns=["Unnamed: 0"])

In [ ]:
import regex as re

In [ ]:
vie = df["UTZ_VIA_DES"].drop_duplicates().reset_index(drop=True)

In [ ]:
vie

In [ ]:
data_vie["vie"] = vie

In [ ]:
data_vie

In [ ]:
indirizzo = data_vie["indirizzo"]

In [ ]:
indir = pd.Series(indirizzo).astype(str)

In [ ]:
indir.apply(lambda x: re.search(" Municipio ", x)).astype(bool).astype("uint8")

I numeri romani sono sempre relativi ai municipi, mi basterebbe estrarli con flashtext

In [ ]:
# pip install flashtext

Cominciamo dai quartieri!

Passiamo ai numeri romani legati al municipio, se inserisco anche 'Municipio Roma ' di sicuro rendo la richiesta più specifica e dunque più precisa.


In [ ]:
mun_list = [
    "Municipio Roma I",
    "Municipio Roma II",
    "Municipio Roma III",
    "Municipio Roma IV",
    "Municipio Roma V",
    "Municipio Roma VI",
    "Municipio Roma VII" "Municipio Roma VIII",
    "Municipio Roma IX",
    "Municipio Roma X",
    "Municipio Roma XI",
    "Municipio Roma XII",
    "Municipio Roma XIII",
    "Municipio Roma XIV",
    "Municipio Roma XV",
]

In [ ]:
from flashtext import KeywordProcessor

keyword_processor = KeywordProcessor()

In [ ]:
keyword_processor = KeywordProcessor(case_sensitive=False)
keyword_processor.add_keywords_from_list(mun_list)  # .to_list()

mun_text = indir.apply(lambda x: keyword_processor.extract_keywords(x)).to_list()

mun_text

In [ ]:
j = 0
for i in range(len(mun_text)):
    if mun_text[i] == []:
        #         print(i)
        j += 1

j #Quindi non ho 6490 Municipi

In [ ]:
for i in range(len(mun_text)):
    if mun_text[i] == []:
        mun_text[i] = np.nan

In [ ]:
mun_text

In [ ]:
data_vie["municipio"] = mun_text

In [ ]:
data_vie = data_vie.dropna().reset_index(drop=True)

In [ ]:
data_vie["vie"]

In [ ]:
ndf = df.loc[df["UTZ_VIA_DES"].isin(data_vie["vie"])].reset_index(drop=True)

In [ ]:
ndf["vie"] = ndf["UTZ_VIA_DES"]

In [ ]:
ndf["vie"]

In [ ]:
data_vie

In [ ]:
data_merge = pd.merge(ndf, data_vie, on="vie", how="left")

In [ ]:
values = np.ones(len(data_merge))

In [ ]:
data_merge["values"] = values

In [ ]:
datavia = pd.DataFrame()

datavia["lat"] = data_merge["lat"]
datavia["long"] = data_merge["long"]
datavia["values"] = data_merge["values"]

datavia

In [ ]:
datavia

In [ ]:
# m = folium.Map(location=[41.9109, 12.4818], tiles="cartodbpositron", zoom_start = 11)
m = folium.Map(location=[41.9109, 12.4818], zoom_start=11)
folium.GeoJson("C:/Users/gmauri01/intellera_code/progetto_comune_roma/municipi.geojson").add_to(m)
m

In [ ]:
# import pandas as pd
# import numpy as np
# import folium
# from folium.plugins import HeatMap
# import matplotlib as mpl


# # for elt in list(zip( datavia.lat , datavia.long, datavia.values)):
# #     folium.Circle(elt[:2], color="white", radius=elt[2]).add_to(m)
# start_time = time.time()


# for elt in list(zip( datavia.lat , datavia.long, datavia.values)):
#     folium.Circle(elt[:2], color="white",radius=0.001
#                  ).add_to(m)
# # df.values used here is a (250, 3) numpy.ndarray
# # with (lat, lon, weight) for each data point
# HeatMap(datavia.values, min_opacity=0.1).add_to(m)
# print("--- %s seconds ---" % (time.time() - start_time))
# m

In [ ]:
# m.save("prova1_dots.html")

In [ ]:
datavia

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
import matplotlib as mpl


# for elt in list(zip( datavia.lat , datavia.long, datavia.values)):
#     folium.Circle(elt[:2], color="white", radius=elt[2]).add_to(m)
start_time = time.time()


# for elt in list(zip( datavia.lat , datavia.long, datavia.values)):
#     folium.Circle(elt[:2], color="white",radius=0.001
#                  ).add_to(m)
# df.values used here is a (250, 3) numpy.ndarray
# with (lat, lon, weight) for each data point
HeatMap(datavia.values, min_opacity=0.1).add_to(m)
print("--- %s seconds ---" % (time.time() - start_time))
m

In [ ]:
# m.save('prova1_no_dots.html')

A questo punto ho i municipi e posso anche sapere quanti punti ho per municipio.

## Quanti punti per municipio?

In [ ]:
data_merge.municipio[0]

In [ ]:
mun = []
for i in range(len(data_merge)):
    mun.append(data_merge.municipio[i][0])

In [ ]:
mun.count("Municipio Roma I")

In [ ]:
mun.count("Municipio Roma II")

In [ ]:
municipi_lista = data_merge["municipio"].drop_duplicates().reset_index(drop=True)

In [ ]:
for i in range(len(municipi_lista)):
    municipi_lista[i] = municipi_lista[i][0]

In [ ]:
municipi_lista

Mancano i municipi 7 ed 8

In [ ]:
municipi_counts = np.zeros(len(municipi_lista))
for i in range(len(municipi_lista)):
    municipi_counts[i] = mun.count(municipi_lista[i])

In [ ]:
municipi_counts

In [ ]:
m_data = pd.DataFrame()

In [ ]:
m_data["municipi"] = municipi_lista
m_data["counts"] = municipi_counts

In [ ]:
m_data

## Adesso dovrei colorare la mappa dei municipi

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import folium

In [ ]:
# FUNCTIONS
# def json_to_geojson(data, districts):
#     # create a geojson from a list of dictionaries
#     # containing coordinates with the name of the polygon
#     # in our case a polygon is a district
#     assert type(data) == list, "The parameter data should be a list of coordinates with a name argument!"

#     geojson = {
#         "type": "FeatureCollection",
#         "features": [
#         {
#             "type": "Feature",
#             "geometry" : {
#                 "type": "Polygon",
#                 "name": district,
#                 "coordinates": [[[d["lon"], d["lat"]] for d in data if d['name'] == district]],
#                 },
#             "properties" : {'name': district},

#          } for district in districts]
#     }

#     return geojson

# def parse_inc_tax_data(income_tax):
#     income_tax.columns = ['district', 'num', 'inc_tax']
#     income_tax = income_tax.dropna()
#     income_tax = income_tax[income_tax['district'].str.contains('kerület')]
#     income_tax['district'] = [dist.replace("kerület", "kerulet").strip() for dist in income_tax['district'].tolist()]

#     for col in ['inc_tax', 'num']:
#         income_tax[col] = [float(f.replace(" ", "")) for f in income_tax[col].tolist()]

#     income_tax['inc_tax_per_capita'] = income_tax['inc_tax'] * 1000 / income_tax['num']
#     income_tax = income_tax.drop(['inc_tax', 'num'], axis = 1)
#     return income_tax

In [ ]:
# load data
# with open('data/district_coords.txt', 'r') as fp:
#     district_coords = json.loads(fp.read())

In [ ]:
# get a list of districts
# districts = list(set([district_coord['name'] for district_coord in district_coords]))

In [ ]:
# create a geojson
# geojson = json_to_geojson(district_coords, districts)

In [ ]:
# geojson='C:/Users/gmauri01/intellera_code/progetto_comune_roma/municipi.geojson'

In [ ]:
# # create geodataframe and add centroids
# gdf = gpd.GeoDataFrame.from_features(geojson)
# gdf['centroid_lon'] = gdf['geometry'].centroid.x
# gdf['centroid_lat'] = gdf['geometry'].centroid.y
# gdf.crs = {'init' :'epsg:4326'}
# gdf.head()

In [ ]:
# # get income tax per capita by districts
# income_tax = pd.read_csv('data/income_tax_data.csv', sep=';')
# income_tax = parse_inc_tax_data(income_tax)
# income_tax.head()

In [ ]:
# merge with inc_tax data
# gdf_income_tax = gdf.merge(income_tax, left_on='name', right_on='district')
# gdf_income_tax.head()
# geometry	name	centroid_lon	centroid_lat	district	inc_tax_per_capita
# 0	POLYGON ((19.0172675 47.532714, 19.0173049 47....	III. kerulet	19.034511	47.569678	III. kerulet	484713.119515
# 1	POLYGON ((19.0345509 47.438895, 19.0352686 47....	XI. kerulet	19.019072	47.460257	XI. kerulet	543127.162188
# 2	POLYGON ((19.1574138 47.4383984, 19.1581105 47...	XVIII. kerulet	19.201757	47.430279	XVIII. kerulet	374651.283307
# 3	POLYGON ((19.0633281 47.5176955, 19.0635358 47...	VI. kerulet	19.065279	47.509549	VI. kerulet	494609.969605
# 4	POLYGON ((19.2207833 47.4925469, 19.2216553 47...	X. kerulet	19.158684	47.483381	X. kerulet	333391.725328

In [ ]:
# CREATE A MAP IN FOLIUM
# # create a map
# m = folium.Map(location=[np.median(gdf_income_tax['centroid_lat'].tolist()), np.median(gdf_income_tax['centroid_lon'].tolist())], tiles='Stamen Toner', zoom_start=12)

In [ ]:
m_data

In [ ]:
import json

In [ ]:
# Opening JSON file
f = open("C:/Users/gmauri01/intellera_code/progetto_comune_roma/municipi.geojson")

data = json.load(f)

In [ ]:
# municipio_geojson={ "type": "Feature",
#                     "bbox": it_geojson["bbox"],
#                     "features": [
#                                elem
#                                for elem in it_geojson["features"]
#                                if elem["properties"]["reg_name"] == nome_regione
#                            ],
#                       }

In [ ]:
# def load_geojson(doc, nome_regione):
#     with open(os.path.join(APP_DATA_PATH, doc)) as f:
#         it_geojson = json.load(f)
#         regione_geojson = {        "type": "Feature",
#                            "bbox": it_geojson["bbox"],
#                            "features": [
#                                elem
#                                for elem in it_geojson["features"]
#                                if elem["properties"]["reg_name"] == nome_regione
#                            ],
#                           }
#         return regione_geojson
#     def province_map(df_annunci_provincia, regione_geojson):
#         map = px.choropleth_mapbox(
#             df_annunci_provincia,
#             geojson=regione_geojson,
#             featureidkey="properties.prov_name",
#             locations="Provincia",
#             color="N° annunci",
#             range_color=(df_annunci_provincia["N° annunci"].min(),
#                          df_annunci_provincia["N° annunci"].max()),
#             color_continuous_scale=px.colors.sequential.Reds,
#             mapbox_style="carto-positron",
#             zoom=5.4,        center={"lat": 43.31, "lon": 11.33},
#             opacity=0.4,        height=330,        width=500,
#             custom_data=[
#                 df_annunci_provincia["Provincia"],

#                 df_annunci_provincia["N° annunci"],
#             ],    )    map.update_traces(
#             hovertemplate="<b>Provincia: </b> %{customdata[0]} <br>"        + "<b>N° annunci: </b> %{customdata[1]:,} <br><extra></extra>",    )    map.update_coloraxes(colorbar_tickformat=",")    map.update_layout(        {"margin": {"t": 0}},        hoverlabel=dict(bgcolor="white", font_size=12, namelength=-1),    )    return map

In [ ]:
# def load_geojson(doc, nome_regione):
#     with open(os.path.join(APP_DATA_PATH, doc)) as f:
#         it_geojson = json.load(f)
#         regione_geojson = {        "type": "FeatureCollection",
#                            "bbox": it_geojson["bbox"],
#                            "features": [
#                                elem
#                                for elem in it_geojson["features"]
#                                if elem["properties"]["reg_name"] == nome_regione
#                            ],
#                           }
#         return regione_geojson

In [ ]:
# data['features'][0]

In [ ]:
# import os

In [ ]:
# 'C:/Users/gmauri01/intellera_code/progetto_comune_roma/

In [ ]:
# ## add chloropleth layer
# folium.Choropleth(
#     geo_data=data,
#     name='visualize',
#     data=m_data,
#     columns=['counts'],
#     fill_color='YlGn',
#     key_on='feature.geometry.coordinates',
#     legend_name='inc'
# ).add_to(m)

In [ ]:
# # add markers with basic information
# fg = folium.FeatureGroup(name='District Info')
# for lat, lon, val, name in zip(gdf_income_tax['centroid_lat'].tolist(), gdf_income_tax['centroid_lon'].tolist(), gdf_income_tax['inc_tax_per_capita'].tolist(), gdf_income_tax['name'].tolist()):
#     html = f"""
#     <h2>{name}<\h2><br>
#     <h4>Income Tax Per Capita: {int(round(val,0))} HUF <\h4>
#     """
#     fg.add_child(folium.Marker(location=[lat, lon], popup=html))

# m.add_child(fg)

# # enable layers to be turned in or out
# folium.LayerControl().add_to(m)

# # save it
# m.save('outputs/income_tax_percapita_by_districts.html')
# m